In [3]:
# 匯入套件
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
import json

In [4]:
'''
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
'''
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")              #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(options = options)

# 螢幕最大化
driver.maximize_window()

# 放置 FB 個人發文的基本資訊
listPost = []

'''
以下定義了一些方法，作為爬蟲流程
'''

# 前往 FB
def begin():    
    # 走訪網址
    driver.get('https://www.facebook.com/')

# 登入
def login():
    # 輸入帳號
    inputEmail = driver.find_element_by_id('email') 
    inputEmail.send_keys("你的帳號")

    # 輸入密碼
    inputPwd = driver.find_element_by_id('pass')
    inputPwd.send_keys("你的密碼")

    # 按下登入/送出鈕
    btnSubmit = driver.find_element_by_id('u_0_b')
    btnSubmit.click()
    
    # 個人頁面連結的選擇器
    cssSelectorMe = 'a[href="/me/"]'

    try:
        # 顯性等待個人發文的元素出現
        WebDriverWait(driver, 30).until( EC.presence_of_element_located( (By.CSS_SELECTOR, cssSelectorMe) ) )
    except TimeoutException:
        print('等待逾時！')

# 走訪個人頁
def visit():
    driver.get('https://www.facebook.com/darreninfo/')

# 捲動頁面
def scroll():
    # 瀏覽器內部的高度
    innerHeightOfWindow = 0
    
    # 當前捲動的量(高度)
    totalOffset = 0

    # 每捲一次，休息幾秒
    sleepingSecond = 1
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while totalOffset <= innerHeightOfWindow:
        # 每次移動高度
        totalOffset += 300;
        
        # 捲動的 js code
        js_scroll = "(function (){{window.scrollTo({{top:{}, behavior: 'smooth' }});}})();".format(totalOffset)
        
        # 執行 js code
        driver.execute_script(js_scroll)
        
        # 強制等待
        sleep(sleepingSecond)
        
        # 透過執行 js 語法來取得捲動後的高度
        innerHeightOfWindow = driver.execute_script('return window.document.documentElement.scrollHeight;');
        
        # 強制等待
        sleep(sleepingSecond)
        
        # 印出捲動距離
        print("innerHeightOfWindow: {}, totalOffset: {}".format(innerHeightOfWindow, totalOffset))
        
        # 為了實驗功能，捲動超過一定的距離，就結束程式
        if totalOffset > 8000:
            break

# 分析元素內容
def parse():
    # 確認當前動個人態牆的發文數量
    cssSelectorPost = 'div[data-ad-comet-preview="message"] span[dir="auto"]'

    try:
        # 顯性等待個人發文的元素出現
        #WebDriverWait(driver, 30).until( EC.visibility_of_element_located( (By.CSS_SELECTOR, cssSelectorPost) ) )

        # 取得元素
        div = driver.find_elements_by_css_selector(cssSelectorPost)

        # 將每個發文的內容擷取出來
        for index, elm in enumerate(div):
            print("index: {}".format(index))
            print("text: {}".format(elm.text))
            print()
            
            # 將資料新增到 list 當中
            listPost.append({
                "index": index,
                "text":elm.text
            })
        
        # 將放置發文的 list，以 JSON 格式存入檔案
        fp = open("FB.json", "w",encoding='UTF-8')
        fp.write( json.dumps(listPost, ensure_ascii=False) )
        fp.close()

    except TimeoutException:
        print('等待逾時！')

# 關閉瀏覽器
def end():
    # 關閉瀏覽器
    driver.quit()

    
    
# 主程式
if __name__ == '__main__':
    begin()
    login()
    visit()
    scroll()
    parse()
    end()

innerHeightOfWindow: 711, totalOffset: 300
innerHeightOfWindow: 4852, totalOffset: 600
innerHeightOfWindow: 4852, totalOffset: 900
innerHeightOfWindow: 5594, totalOffset: 1200
innerHeightOfWindow: 5594, totalOffset: 1500
innerHeightOfWindow: 6106, totalOffset: 1800
innerHeightOfWindow: 6106, totalOffset: 2100
innerHeightOfWindow: 7554, totalOffset: 2400
innerHeightOfWindow: 7554, totalOffset: 2700
innerHeightOfWindow: 7554, totalOffset: 3000
innerHeightOfWindow: 7554, totalOffset: 3300
innerHeightOfWindow: 7554, totalOffset: 3600
innerHeightOfWindow: 8894, totalOffset: 3900
innerHeightOfWindow: 8894, totalOffset: 4200
innerHeightOfWindow: 8894, totalOffset: 4500
innerHeightOfWindow: 8894, totalOffset: 4800
innerHeightOfWindow: 9474, totalOffset: 5100
innerHeightOfWindow: 9474, totalOffset: 5400
innerHeightOfWindow: 11782, totalOffset: 5700
innerHeightOfWindow: 11782, totalOffset: 6000
innerHeightOfWindow: 11782, totalOffset: 6300
innerHeightOfWindow: 11782, totalOffset: 6600
innerHeigh